In [ ]:
import pandas as pd
import os
import time
import csv
import re

In [ ]:

raw_data_dir = 'dataset'
cleaned_data_dir = 'datasets_cleaned'
file_to_clean_name = 'etd_ugm.csv'

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")


In [ ]:
def clean_and_quote_csv(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    # Baca file CSV
    df = pd.read_csv(input_filepath)
    
    # Replace karakter khusus
    df = df.replace("–", "-", regex=True)
    
    # Pastikan judul diapit tanda petik dua (hanya jika belum ada)
    if 'judul' in df.columns:
        df['judul'] = df['judul'].apply(
            lambda x: f'"{x}"' if not (str(x).startswith('"') and str(x).endswith('"')) else x
        )
    
    # Simpan file dengan encoding UTF-8
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print("\nProses pembersihan selesai!")
    print(f"File bersih disimpan di: {output_filepath}")

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")

# Jalankan proses pembersihan
clean_and_quote_csv(input_filepath, output_filepath)

In [ ]:
def clean_and_quote_csv(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    # Baca file CSV
    df = pd.read_csv(input_filepath)
    
    # Replace karakter khusus
    df = df.replace("–", "-", regex=True)
    
    # Pastikan judul diapit tanda petik dua (hanya jika belum ada)
    if 'judul' in df.columns:
        df['judul'] = df['judul'].apply(
            lambda x: f'"{x}"' if not (str(x).startswith('"') and str(x).endswith('"')) else x
        )
    
    # Simpan file dengan encoding UTF-8
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print("\nProses pembersihan selesai!")
    print(f"File bersih disimpan di: {output_filepath}")

# Konfigurasi path
raw_data_dir = 'dataset'
cleaned_data_dir = 'datasets_cleaned'
file_to_clean_name = 'etd_usk.csv'

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")

# Jalankan proses pembersihan
clean_and_quote_csv(input_filepath, output_filepath)

`Formating untuk Dataset_ugm`

In [ ]:
def fix_all_formats(input_filepath, output_filepath):
    """
    Menangani semua format yang tidak konsisten dalam data ETD
    """
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    data = []
    
    with open(input_filepath, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    
    # Split berdasarkan pattern yang mungkin
    # Pattern 1: Judul,"Konten" (dengan newline di antara entries)
    entries = re.split(r'\n(?=[^",\n]+,")', content)
    
    # Jika tidak berhasil, coba pattern lain
    if len(entries) <= 1:
        entries = re.split(r'\n"(?=[^"]+","[^"])', content)
    
    for entry in entries:
        entry = entry.strip()
        if not entry or 'judul' in entry.lower():
            continue
        
        # Handle berbagai format
        judul, konten = parse_entry(entry)
        
        if judul and konten:
            # Cleaning
            judul = clean_text(judul)
            konten = clean_text(konten)
            
            data.append({'judul': judul, 'konten': konten})
    
    # Buat DataFrame
    df = pd.DataFrame(data)
    
    # Simpan dengan format CSV standar
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print(f"✅ Formatting selesai!")
    print(f"   Data diproses: {len(data)} baris")
    print(f"   File disimpan: {output_filepath}")
    
    # Tampilkan sample
    if len(df) > 0:
        print(f"\n📋 Sample hasil:")
        print("=" * 80)
        for i in range(min(3, len(df))):
            print(f"Data {i+1}:")
            print(f"  Judul: {df['judul'].iloc[i][:80]}...")
            print(f"  Konten: {df['konten'].iloc[i][:80]}...")
            print()

def parse_entry(entry):
    """
    Parse satu entry dengan berbagai format yang mungkin
    """
    entry = entry.strip()
    
    # Format 1: Judul,"Konten" (normal)
    if re.match(r'^[^"]+,"[^"]+', entry):
        first_quote = entry.find('"')
        judul = entry[:first_quote-1].strip() if first_quote > 0 else ""
        konten = entry[first_quote:].strip()
        
        # Hapus quotes dari konten
        if konten.startswith('"') and konten.endswith('"'):
            konten = konten[1:-1]
        elif konten.startswith('"'):
            konten = konten[1:]
        elif konten.endswith('"'):
            konten = konten[:-1]
            
        return judul, konten
    
    # Format 2: "Judul","Konten" (dengan quotes di judul juga)
    elif re.match(r'^"[^"]+","[^"]+', entry):
        parts = entry.split('","', 1)
        if len(parts) == 2:
            judul = parts[0][1:].strip() if parts[0].startswith('"') else parts[0].strip()
            konten = parts[1][:-1].strip() if parts[1].endswith('"') else parts[1].strip()
            return judul, konten
    
    # Format 3: Triple quotes """Judul","Konten"""
    elif entry.startswith('"""') and entry.endswith('"""'):
        entry = entry[3:-3]  # Hapus triple quotes
        return parse_entry(entry)  # Parse ulang
    
    # Format 4: Judul,"""Konten"""
    elif ',"""' in entry and entry.endswith('"""'):
        parts = entry.split(',"""', 1)
        if len(parts) == 2:
            judul = parts[0].strip()
            konten = parts[1][:-3].strip()  # Hapus triple quotes akhir
            return judul, konten
    
    # Format 5: Cari koma pertama sebagai pemisah
    elif ',' in entry:
        first_comma = entry.find(',')
        judul = entry[:first_comma].strip()
        konten = entry[first_comma+1:].strip()
        
        # Hapus quotes berlebihan
        judul = judul.strip('"')
        konten = konten.strip('"')
        
        return judul, konten
    
    return None, None

def clean_text(text):
    """
    Membersihkan teks dari format yang tidak diinginkan
    """
    if not text:
        return ""
    
    # Hapus triple quotes
    text = re.sub(r'"+', '"', text)
    
    # Hapus quotes di awal dan akhir jika berlebihan
    text = text.strip()
    while text.startswith('"') and text.endswith('"'):
        text = text[1:-1].strip()
    
    # Replace karakter khusus
    text = text.replace("–", "-")
    text = text.replace("”", '"')
    text = text.replace("“", '"')
    
    # Normalisasi whitespace
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def process_complex_file(input_filepath, output_filepath):
    """
    Process file dengan pendekatan line-by-line untuk data yang kompleks
    """
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    data = []
    current_entry = []
    
    with open(input_filepath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            
            if not line or 'judul' in line.lower():
                continue
            
            # Jika line mengandung pattern judul baru, proses entry sebelumnya
            if re.match(r'^[^",]+,"[^"]+', line) and current_entry:
                process_current_entry(current_entry, data)
                current_entry = []
            
            current_entry.append(line)
        
        # Process entry terakhir
        if current_entry:
            process_current_entry(current_entry, data)
    
    # Buat DataFrame
    df = pd.DataFrame(data)
    
    # Simpan dengan format CSV standar
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print(f"✅ Complex formatting selesai!")
    print(f"   Data diproses: {len(data)} baris")
    print(f"   File disimpan: {output_filepath}")

def process_current_entry(entry_lines, data):
    """
    Process kumpulan lines menjadi satu entry
    """
    full_entry = ' '.join(entry_lines)
    judul, konten = parse_entry(full_entry)
    
    if judul and konten:
        judul = clean_text(judul)
        konten = clean_text(konten)
        data.append({'judul': judul, 'konten': konten})

# Path sesuai yang kamu mau
input_filepath = os.path.join('dataset', 'etd_ugm.csv')
output_filepath = os.path.join('datasets_cleaned', 'etd_ugm.csv')

print("Memulai proses formatting...")
print(f"  Input : {input_filepath}")
print(f"  Output: {output_filepath}")

# Coba metode yang berbeda
try:
    fix_all_formats(input_filepath, output_filepath)
except Exception as e:
    print(f"❌ Method 1 failed: {e}")
    print("Trying complex method...")
    try:
        process_complex_file(input_filepath, output_filepath)
    except Exception as e2:
        print(f"❌ All methods failed: {e2}")

In [ ]:
def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks
    """
    if not text:
        return text
    
    text = text.strip()
    
    # Handle triple quotes
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3]
    # Handle double quotes
    elif text.startswith('""') and text.endswith('""'):
        text = text[2:-2]
    # Handle single quote
    elif text.startswith('"') and text.endswith('"'):
        text = text[1:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    pattern = r'^([\w\s.,]+?)(\–|\-)\s*'
    cleaned_text = re.sub(pattern, '', text, count=1)
    return cleaned_text

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader
    try:
        reader = csv.reader([line])
        row = next(reader)
        if len(row) == 2:
            return row
    except:
        pass
    
    # Approach 2: Manual parsing dengan '","'
    if '","' in line:
        parts = line.split('","', 1)
        if len(parts) == 2:
            judul = parts[0].strip().strip('"')
            konten = parts[1].strip().strip('"')
            return [judul, konten]
    
    return None

def clean_and_split_csv(input_path, output_path):
    """
    Proses utama pembersihan CSV
    """
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")
    
    success_count = 0
    fail_count = 0
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower() == 'judul,konten':
                continue
            
            # Parsing line
            parsed = parse_csv_line_robust(line)
            
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                
                # Bersihkan kutip berlebihan
                judul = clean_excessive_quotes(judul)
                konten = clean_excessive_quotes(konten)
                
                # Hapus prefix dari konten
                konten = remove_source_prefix(konten)
                
                # Potong konten agar tidak terlalu panjang untuk testing
                if len(konten) > 200:
                    konten = konten[:200] + "..."
                
                writer.writerow([judul, konten])
                success_count += 1
                    
            else:
                fail_count += 1
    
    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")

def create_test_version(input_path, output_test_path):
    """
    Buat versi test dengan hanya 5 baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat versi test (5 baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower() == 'judul,konten':
                continue
            
            if success_count >= 5:  # Hanya ambil 5 baris
                break
            
            # Parsing line
            parsed = parse_csv_line_robust(line)
            
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                
                # Bersihkan kutip berlebihan
                judul = clean_excessive_quotes(judul)
                konten = clean_excessive_quotes(konten)
                
                # Hapus prefix dari konten
                konten = remove_source_prefix(konten)
                
                # Potong konten agar pendek
                if len(konten) > 100:
                    konten = konten[:100] + "..."
                
                writer.writerow([judul, konten])
                success_count += 1
                
                print(f"Test Baris {success_count}:")
                print(f"  Judul: {judul}")
                print(f"  Konten: {konten}")
                print()

def validate_and_display(file_path):
    """
    Validasi dan tampilkan dengan cara yang lebih jelas
    """
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        
        print(f"\n🎯 DETAIL PER KOLOM:")
        print(f"   judul - Panjang: {len(df['judul'].iloc[0])} karakter")
        print(f"   konten - Panjang: {len(df['konten'].iloc[0])} karakter")
        
        print(f"\n📊 SAMPLE DATA (dengan border):")
        print("=" * 80)
        for i in range(min(3, len(df))):
            print(f"ROW {i+1}:")
            print(f"┌{'─' * 78}┐")
            print(f"│ JUDUL  : {df['judul'].iloc[i][:70]:70} │")
            print(f"├{'─' * 78}┤")
            print(f"│ KONTEN : {df['konten'].iloc[i][:70]:70} │")
            print(f"└{'─' * 78}┘")
            print()
            
        return True
        
    except Exception as e:
        print(f"❌ ERROR: {e}")
        return False

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'mojok.csv')
    output_filepath = os.path.join('cleaned', 'mojok_cleaned.csv')
    output_test_path = os.path.join('cleaned', 'mojok_TEST.csv')  # File test kecil
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
    else:
        # 1. Buat versi test kecil dulu
        create_test_version(input_filepath, output_test_path)
        
        # 2. Validasi versi test
        print("\n" + "="*50)
        print("VALIDASI VERSI TEST (5 BARIS):")
        print("="*50)
        validate_and_display(output_test_path)
        
        # 3. Jika test berhasil, buat versi lengkap
        print("\n" + "="*50)
        print("MEMBUAT VERSI LENGKAP:")
        print("="*50)
        clean_and_split_csv(input_filepath, output_filepath)
        
        # 4. Validasi versi lengkap
        print("\n" + "="*50)
        print("VALIDASI VERSI LENGKAP:")
        print("="*50)
        validate_and_display(output_filepath)
        
        # 5. Final check
        print("\n🎯 FINAL VERIFICATION:")
        df = pd.read_csv(output_test_path)
        print("Data BERHASIL dipisah dengan benar!")
        print("Jika di notebook warnanya sama, itu hanya masalah tampilan pandas.")
        print("Coba buka file CSV langsung di Excel/text editor untuk verifikasi.")

## Cleanup Dataset

`Mojok-Dataset` - `Fix`

In [ ]:
def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks dengan lebih agresif
    """
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    # Hapus semua kutip berlebihan di awal dan akhir
    # Hapus kutip berulang di awal
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    # Hapus kutip berulang di akhir
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    if not text:
        return text
    
    # Pattern untuk berbagai variasi penulisan KOMPAS.com
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    # Bersihkan kutip ganda berlebihan
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust untuk handle berbagai format
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader dengan quote handling yang lebih baik
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    # Approach 2: Manual parsing dengan pattern matching
    # Pattern untuk: "judul","konten"
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    # Approach 3: Handle kasus dengan kutip berlebihan
    if line.count('"') >= 4:
        # Cari pemisah antara judul dan konten
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  # Tambahkan kembali kutip yang hilang
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    # Approach 4: Split sederhana pada koma pertama
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    """
    Pastikan judul dan konten memiliki format kutip yang benar
    untuk output CSV
    """
    # Bersihkan dulu dari kutip berlebihan
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    # Hapus prefix dari konten
    konten = remove_source_prefix(konten)
    
    # Pastikan tidak ada kutip berlebihan di akhir
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # QUOTE_ALL akan otomatis menambahkan ""
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            # Skip baris kosong atau header
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            # Debug untuk line dengan banyak kutip
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            # Coba parsing robust terlebih dahulu
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            # Jika parsing robust gagal, coba approach lainnya
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        # Tampilkan format sebenarnya yang disimpan
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  # Tampilkan 4 baris pertama termasuk header
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    """
    Buat file test dengan beberapa baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            # Debug info - PERBAIKAN: gunakan repr() untuk menghindari masalah quote
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  # chr(34) adalah "
            
            # Parsing
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'mojok.csv')
    output_test_path = os.path.join('cleaned', 'mojok_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/mojok.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 9454)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Kompas-Dataset` - `Fix`

In [ ]:
import re
import csv
import os
import pandas as pd

def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks dengan lebih agresif
    """
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    # Hapus semua kutip berlebihan di awal dan akhir
    # Hapus kutip berulang di awal
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    # Hapus kutip berulang di akhir
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    if not text:
        return text
    
    # Pattern untuk berbagai variasi penulisan KOMPAS.com
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    # Bersihkan kutip ganda berlebihan
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust untuk handle berbagai format
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader dengan quote handling yang lebih baik
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    # Approach 2: Manual parsing dengan pattern matching
    # Pattern untuk: "judul","konten"
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    # Approach 3: Handle kasus dengan kutip berlebihan
    if line.count('"') >= 4:
        # Cari pemisah antara judul dan konten
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  # Tambahkan kembali kutip yang hilang
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    # Approach 4: Split sederhana pada koma pertama
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    """
    Pastikan judul dan konten memiliki format kutip yang benar
    untuk output CSV
    """
    # Bersihkan dulu dari kutip berlebihan
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    # Hapus prefix dari konten
    konten = remove_source_prefix(konten)
    
    # Pastikan tidak ada kutip berlebihan di akhir
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # QUOTE_ALL akan otomatis menambahkan ""
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            # Skip baris kosong atau header
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            # Debug untuk line dengan banyak kutip
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            # Coba parsing robust terlebih dahulu
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            # Jika parsing robust gagal, coba approach lainnya
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        # Tampilkan format sebenarnya yang disimpan
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  # Tampilkan 4 baris pertama termasuk header
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    """
    Buat file test dengan beberapa baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            # Debug info - PERBAIKAN: gunakan repr() untuk menghindari masalah quote
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  # chr(34) adalah "
            
            # Parsing
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'kompas.csv')
    output_test_path = os.path.join('cleaned', 'kompas_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/kompas.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 9099)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Tempo Dataset` - `Fix`


In [ ]:
def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks dengan lebih agresif
    """
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    # Hapus semua kutip berlebihan di awal dan akhir
    # Hapus kutip berulang di awal
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    # Hapus kutip berulang di akhir
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    if not text:
        return text
    
    # Pattern untuk berbagai variasi penulisan KOMPAS.com
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    # Bersihkan kutip ganda berlebihan
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust untuk handle berbagai format
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader dengan quote handling yang lebih baik
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    # Approach 2: Manual parsing dengan pattern matching
    # Pattern untuk: "judul","konten"
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    # Approach 3: Handle kasus dengan kutip berlebihan
    if line.count('"') >= 4:
        # Cari pemisah antara judul dan konten
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  # Tambahkan kembali kutip yang hilang
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    # Approach 4: Split sederhana pada koma pertama
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    """
    Pastikan judul dan konten memiliki format kutip yang benar
    untuk output CSV
    """
    # Bersihkan dulu dari kutip berlebihan
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    # Hapus prefix dari konten
    konten = remove_source_prefix(konten)
    
    # Pastikan tidak ada kutip berlebihan di akhir
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # QUOTE_ALL akan otomatis menambahkan ""
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            # Skip baris kosong atau header
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            # Debug untuk line dengan banyak kutip
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            # Coba parsing robust terlebih dahulu
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            # Jika parsing robust gagal, coba approach lainnya
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        # Tampilkan format sebenarnya yang disimpan
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  # Tampilkan 4 baris pertama termasuk header
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    """
    Buat file test dengan beberapa baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            # Debug info - PERBAIKAN: gunakan repr() untuk menghindari masalah quote
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  # chr(34) adalah "
            
            # Parsing
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'tempo.csv')
    output_test_path = os.path.join('cleaned', 'tempo_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/tempo.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 10000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Ugm Dataset` - `fix`

In [ ]:
def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks dengan lebih agresif
    """
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    # Hapus semua kutip berlebihan di awal dan akhir
    # Hapus kutip berulang di awal
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    # Hapus kutip berulang di akhir
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    if not text:
        return text
    
    # Pattern untuk berbagai variasi penulisan KOMPAS.com
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    # Bersihkan kutip ganda berlebihan
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust untuk handle berbagai format
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader dengan quote handling yang lebih baik
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    # Approach 2: Manual parsing dengan pattern matching
    # Pattern untuk: "judul","konten"
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    # Approach 3: Handle kasus dengan kutip berlebihan
    if line.count('"') >= 4:
        # Cari pemisah antara judul dan konten
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  # Tambahkan kembali kutip yang hilang
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    # Approach 4: Split sederhana pada koma pertama
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    """
    Pastikan judul dan konten memiliki format kutip yang benar
    untuk output CSV
    """
    # Bersihkan dulu dari kutip berlebihan
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    # Hapus prefix dari konten
    konten = remove_source_prefix(konten)
    
    # Pastikan tidak ada kutip berlebihan di akhir
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # QUOTE_ALL akan otomatis menambahkan ""
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            # Skip baris kosong atau header
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            # Debug untuk line dengan banyak kutip
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            # Coba parsing robust terlebih dahulu
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            # Jika parsing robust gagal, coba approach lainnya
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        # Tampilkan format sebenarnya yang disimpan
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    """
    Buat file test dengan beberapa baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            # Debug info - PERBAIKAN: gunakan repr() untuk menghindari masalah quote
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  # chr(34) adalah "
            
            # Parsing
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'etd_ugm.csv')
    output_test_path = os.path.join('cleaned', 'etd_ugm_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/etd_ugm.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 50000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Dataset etd_usk` - `Fix`

In [ ]:
def clean_excessive_quotes(text):
    """
    Bersihkan kutip berlebihan dari teks dengan lebih agresif
    """
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    # Hapus semua kutip berlebihan di awal dan akhir
    # Hapus kutip berulang di awal
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    # Hapus kutip berulang di akhir
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    """
    Hapus pola awalan sumber berita
    """
    if not text:
        return text
    
    # Pattern untuk berbagai variasi penulisan KOMPAS.com
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    # Bersihkan kutip ganda berlebihan
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    """
    Parsing line CSV yang robust untuk handle berbagai format
    """
    line = line.strip()
    
    # Approach 1: Gunakan csv.reader dengan quote handling yang lebih baik
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    # Approach 2: Manual parsing dengan pattern matching
    # Pattern untuk: "judul","konten"
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    # Approach 3: Handle kasus dengan kutip berlebihan
    if line.count('"') >= 4:
        # Cari pemisah antara judul dan konten
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  # Tambahkan kembali kutip yang hilang
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    # Approach 4: Split sederhana pada koma pertama
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    """
    Pastikan judul dan konten memiliki format kutip yang benar
    untuk output CSV
    """
    # Bersihkan dulu dari kutip berlebihan
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    # Hapus prefix dari konten
    konten = remove_source_prefix(konten)
    
    # Pastikan tidak ada kutip berlebihan di akhir
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # QUOTE_ALL akan otomatis menambahkan ""
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            # Skip baris kosong atau header
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            # Debug untuk line dengan banyak kutip
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            # Coba parsing robust terlebih dahulu
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            # Jika parsing robust gagal, coba approach lainnya
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        # Tampilkan format sebenarnya yang disimpan
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  # Tampilkan 4 baris pertama termasuk header
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    """
    Buat file test dengan beberapa baris pertama untuk debugging
    """
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            # Debug info - PERBAIKAN: gunakan repr() untuk menghindari masalah quote
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  # chr(34) adalah "
            
            # Parsing
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

# --- Jalankan ---
if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'etd_usk.csv')
    output_test_path = os.path.join('cleaned', 'etd_usk_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/tempo.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 10000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

## Merge Dataset

In [5]:
# Direktori tempat dataset asli Anda berada
input_dir = 'cleaned'

# Direktori untuk menyimpan hasil gabungan
output_dir = 'merge_datasets'

# Daftar nama file yang akan digabungkan
files_to_merge = [
    'etd_ugm_cleaned.csv', 
    'etd_usk_cleaned.csv', 
    'kompas_cleaned.csv', 
    'mojok_cleaned.csv', 
    'tempo_cleaned.csv'
]

# Nama file akhir setelah semua dataset digabungkan
output_filename = os.path.join(output_dir, '0_mergeDataset.csv')

# Ukuran chunk (kalau file besar banget)
chunk_size = 80000

print("Konfigurasi selesai:")
print(f"Input direktori: {input_dir}")
print(f"Output direktori: {output_dir}")
print(f"File output: {output_filename}")
print("="*60)

# Pastikan direktori output ada
os.makedirs(output_dir, exist_ok=True)

# DataFrame kosong untuk menampung hasil gabungan
merged_df = pd.DataFrame()

# Proses penggabungan
for file_name in files_to_merge:
    file_path = os.path.join(input_dir, file_name)
    
    if not os.path.exists(file_path):
        print(f"❌ File tidak ditemukan: {file_name}")
        continue

    print(f"🔹 Membaca {file_name} ...")
    
    # Baca file CSV per chunk (aman untuk file besar)
    try:
        chunks = pd.read_csv(file_path, chunksize=chunk_size)
        for chunk in chunks:
            merged_df = pd.concat([merged_df, chunk], ignore_index=True)
    except Exception as e:
        print(f"⚠️ Gagal membaca {file_name}: {e}")
        continue

print("="*60)
print(f"✅ Total data gabungan: {merged_df.shape[0]} baris, {merged_df.shape[1]} kolom")

# Simpan hasil gabungan ke file CSV baru
merged_df.to_csv(output_filename, index=False, quoting=1, encoding='utf-8')

print(f"💾 Hasil gabungan disimpan di: {output_filename}")


Konfigurasi selesai:
Input direktori: cleaned
Output direktori: merge_datasets
File output: merge_datasets\0_mergeDataset.csv
🔹 Membaca etd_ugm_cleaned.csv ...
🔹 Membaca etd_usk_cleaned.csv ...
🔹 Membaca kompas_cleaned.csv ...


🔹 Membaca mojok_cleaned.csv ...
🔹 Membaca tempo_cleaned.csv ...
✅ Total data gabungan: 46531 baris, 2 kolom
💾 Hasil gabungan disimpan di: merge_datasets\0_mergeDataset.csv
